# Gateway IAM ロールインバウンド認証で AWS Lambda を MCP 化
## Bedrock AgentCore Gateway で AWS Lambda 関数を安全な MCP ツールに変換

## 概要
Bedrock AgentCore Gateway は、既存の AWS Lambda 関数をインフラやホスティングを管理することなく、フルマネージドな MCP サーバーに変換する方法を提供します。Gateway はこれらすべてのツールに対して統一された Model Context Protocol (MCP) インターフェースを提供します。Gateway は、受信リクエストとターゲットリソースへのアウトバウンド接続の両方に対して安全なアクセス制御を確保するために、デュアル認証モデルを採用しています。このフレームワークは2つの主要コンポーネントで構成されています：インバウンド認証（Gateway ターゲットにアクセスしようとするユーザーを検証・認可）とアウトバウンド認証（認証されたユーザーに代わって Gateway がバックエンドリソースに安全に接続することを可能にする）です。Gateway は AWS Lambda 関数への呼び出しを認可するためにアウトバウンド認証として IAM ロールを使用します。

この例では、インバウンドとアウトバウンドの両方の認証に IAM ロールを使用する方法を示します。

![動作の仕組み](images/lambda-gw-iam-inbound.png)

### チュートリアル詳細


| 情報                 | 詳細                                                       |
|:---------------------|:----------------------------------------------------------|
| チュートリアルタイプ  | インタラクティブ                                           |
| AgentCore コンポーネント | AgentCore Gateway                                       |
| エージェントフレームワーク | Strands Agents                                        |
| Gateway ターゲットタイプ | AWS Lambda                                              |
| インバウンド認証      | AWS IAM                                                   |
| アウトバウンド認証    | AWS IAM                                                   |
| LLM モデル           | Anthropic Claude Haiku 4.5、Amazon Nova Pro               |
| チュートリアル構成    | AgentCore Gateway の作成と呼び出し                         |
| チュートリアル分野    | クロスバーティカル                                         |
| 例の複雑さ           | 簡単                                                       |
| 使用 SDK             | boto3                                                     |

チュートリアルの最初の部分では、Lambda 用の AgentCore Gateway ターゲットを作成します

### チュートリアルアーキテクチャ
このチュートリアルでは、AWS Lambda 関数で定義された操作を MCP ツールに変換し、Bedrock AgentCore Gateway でホストします。AWS Sigv4 ヘッダー内の AWS IAM 認証情報を使用したイングレス認証を示します。
デモンストレーション目的で、Amazon Bedrock モデルを使用した Strands Agent を使用します。
この例では、get_order と update_order の2つのツールを持つ非常にシンプルなエージェントを使用します。

## 前提条件

このチュートリアルを実行するには以下が必要です：
* Jupyter notebook（Python カーネル）
* uv
* AWS 認証情報
* AWS コンソール経由での Nova Pro へのアクセス
* Amazon Bedrock AgentCore SDK
* Strands Agents

## AgentCore Gateway 受信リクエストの認証設定
AgentCore Gateway は、インバウンドとアウトバウンドの認証を通じて安全な接続を提供します。インバウンド認証では、AgentCore Gateway は OAuth に加えて AWS IAM 認証情報/ID をサポートして Gateway を呼び出すことができるようになりました。ツールが外部リソースにアクセスする必要がある場合、AgentCore Gateway は API キー、IAM、または OAuth トークンを使用したアウトバウンド認証により、外部リソースへのアクセスを許可または拒否できます。

インバウンド認可フローでは、エージェントまたは MCP クライアントが AWS Signature V4 署名付きリクエストを使用して認証を行い、AgentCore Gateway へのアクセスのための IAM 権限に対して認可を行います。その後、AgentCore Gateway が AWS IAM 認証情報/ID を検証し、インバウンド認可を実行します。

AgentCore Gateway で実行されているツールが外部リソースにアクセスする必要がある場合、IAM ロールが Gateway ターゲットのダウンストリームリソースの認証情報を取得します。AgentCore Gateway は認可資格情報を呼び出し元に渡し、ダウンストリーム API へのアクセスを取得します。

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

In [ ]:
# Set AWS credentials if not using Amazon SageMaker notebook
import os

#os.environ['AWS_ACCESS_KEY_ID']=''
#os.environ['AWS_SECRET_ACCESS_KEY']=''
os.environ['AWS_DEFAULT_REGION']='us-west-2' # set the AWS region




In [ ]:
import os
import sys

# Get the directory of the current script
if '__file__' in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    current_dir = os.getcwd()  # Fallback if __file__ is not defined (e.g., Jupyter)

# Navigate to the directory containing utils.py (one level up)
utils_dir = os.path.abspath(os.path.join(current_dir, '..'))

# Add to sys.path
sys.path.insert(0, utils_dir)



In [ ]:
# Now you can import utils
import utils
#### Create a sample AWS Lambda function that you want to convert into MCP tools
lambda_resp = utils.create_gateway_lambda("lambda_function_code.zip")
if lambda_resp is not None:
    if lambda_resp['exit_code'] == 0:
        print("Lambda 関数が作成されました。ARN: ", lambda_resp['lambda_function_arn'])
    else:
        print("Lambda 関数の作成に失敗しました。メッセージ: ", lambda_resp['lambda_function_arn'])

In [ ]:
#### Create an IAM role for the Gateway to assume
import utils
agentcore_gateway_iam_role = utils.create_agentcore_gateway_role("sample-lambdagateway")
print("Agentcore Gateway ロール ARN: ", agentcore_gateway_iam_role['Role']['Arn'])

# インバウンド認可用に Amazon IAM Authorizer を使用して Gateway を作成

In [ ]:
import time
import boto3
# CreateGateway with Amazon IAM. 
gateway_client = boto3.client('bedrock-agentcore-control', region_name = os.environ['AWS_DEFAULT_REGION'])

create_response = gateway_client.create_gateway(name='TestGWforLambdaIAM',
    roleArn = agentcore_gateway_iam_role['Role']['Arn'], # The IAM Role must have permissions to create/list/get/delete Gateway 
    protocolType='MCP',
    authorizerType='AWS_IAM',
    description='AgentCore Gateway with AWS Lambda target type using Amazon IAM for ingress auth'
)
print(create_response)
# Retrieve the GatewayID used for GatewayTarget creation
gatewayID = create_response["gatewayId"]
gatewayURL = create_response["gatewayUrl"]
print(gatewayID)
time.sleep(10)

# AWS Lambda ターゲットを作成し、MCP ツールに変換

In [ ]:
# Replace the AWS Lambda function ARN below
lambda_target_config = {
    "mcp": {
        "lambda": {
            "lambdaArn": lambda_resp['lambda_function_arn'], # Replace this with your AWS Lambda function ARN
            "toolSchema": {
                "inlinePayload": [
                    {
                        "name": "get_order_tool",
                        "description": "tool to get the order",
                        "inputSchema": {
                            "type": "object",
                            "properties": {
                                "orderId": {
                                    "type": "string"
                                }
                            },
                            "required": ["orderId"]
                        }
                    },                    
                    {
                        "name": "update_order_tool",
                        "description": "tool to update the orderId",
                        "inputSchema": {
                            "type": "object",
                            "properties": {
                                "orderId": {
                                    "type": "string"
                                }
                            },
                            "required": ["orderId"]
                        }
                    }
                ]
            }
        }
    }
}

credential_config = [ 
    {
        "credentialProviderType" : "GATEWAY_IAM_ROLE"
    }
]
targetname='LambdaUsingSDK'
response = gateway_client.create_gateway_target(
    gatewayIdentifier=gatewayID,
    name=targetname,
    description='Lambda Target using SDK',
    targetConfiguration=lambda_target_config,
    credentialProviderConfigurations=credential_config)

# Gateway を呼び出すための AWS IAM ロールを作成

以下の関数は、AWS Bedrock AgentCore が指定された Gateway を呼び出すことを許可する IAM ロールを作成または更新します。指定された Gateway ID に対する bedrock-agentcore:InvokeGateway 権限を付与するインラインポリシーを構築してアタッチします。また、Bedrock AgentCore サービスと呼び出し元の IAM エンティティ（current_arn）の両方がこのロールを引き受けることを許可する信頼ポリシーを設定します。

In [ ]:
#### Create an IAM role to Invoke Gateway
current_role_arn = utils.get_current_role_arn()
print("現在のロール ARN: ", current_role_arn)

agentcore_gateway_iam_invoke_role = utils.create_gateway_invoke_tool_role("gateway-invoke-role",gatewayID , current_role_arn)
print("Agentcore Gateway を呼び出すためのロール ARN: ", agentcore_gateway_iam_invoke_role['Role']['Arn'])

# Strands エージェントが Bedrock AgentCore Gateway を使用して AWS Lambda の MCP ツールを呼び出す

#### MCP クライアント SDK での AWS IAM 認証サポート

AWS IAM 認証が AgentCore Gateway へのインバウンドリクエストでサポートされるようになりましたが、現在のオープンソース MCP クライアント SDK は、特にストリーマブル HTTP 接続に対する SigV4 認証のサポートが限られていることに注意が必要です。しかし、AWS は「AWS Lambda で Model Context Protocol (MCP) サーバーを実行する」プロジェクトを通じてソリューションを提供しており、ストリーミング HTTP 接続での SigV4 認証のための重要な拡張機能が含まれています。

この実装は [AWS Labs GitHub リポジトリ](https://github.com/awslabs/run-model-context-protocol-servers-with-aws-lambda/tree/main) で入手可能で、ストリーミング接続の認証ギャップを埋め、Strands や LangChain などの人気のあるエージェントフレームワークとシームレスに統合できます。StreamableHTTPTransportWithSigV4 クラスは、ストリーミング機能を維持しながら AWS SigV4 署名を処理するように標準 MCP トランスポート層を拡張し、AgentCore Gateway の新しい IAM 認証機能と互換性があります。

In [ ]:
!pip3 install --upgrade strands-agents strands-agents-tools
from strands.models import BedrockModel

## The IAM credentials configured in ~/.aws/credentials should have access to Bedrock model
yourmodel = BedrockModel(
    model_id="us.amazon.nova-pro-v1:0",
    temperature=0.7,
)

In [ ]:
from strands import Agent
import logging
from strands import Agent
import logging
from strands.tools.mcp.mcp_client import MCPClient
from mcp.client.streamable_http import streamablehttp_client 
from botocore.credentials import Credentials
from streamable_http_sigv4 import (
    streamablehttp_client_with_sigv4,
)

SERVICE="bedrock-agentcore"

# Configure the root strands logger. Change it to DEBUG if you are debugging the issue.
logging.getLogger("strands").setLevel(logging.INFO)

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

def create_streamable_http_transport(mcp_url: str, access_token: str):
       return streamablehttp_client(mcp_url, headers={"Authorization": f"Bearer {access_token}"})

def create_streamable_http_transport_sigv4(mcp_url: str, key: str, secret: str, sessionToken: str, serviceName: str, awsRegion: str):
        iamcredentials = Credentials(
            access_key=key,
            secret_key=secret,
            token = sessionToken
        )
        return streamablehttp_client_with_sigv4(
            url=mcp_url,
            credentials=iamcredentials,
            service=serviceName,
            region=awsRegion,
        )

def get_full_tools_list(client):
    more_tools = True
    tools = []
    pagination_token = None
    while more_tools:
        tmp_tools = client.list_tools_sync(pagination_token=pagination_token)
        tools.extend(tmp_tools)
        if tmp_tools.pagination_token is None:
            more_tools = False
        else:
            more_tools = True 
            pagination_token = tmp_tools.pagination_token
    return tools

def call_tool_sync(client, tool_id,tool_name, parameters=None):
    # Call the tool (no pagination argument supported)
    response = client.call_tool_sync(
        tool_use_id=tool_id,
        name=tool_name,
        arguments=parameters
    )

    # Extract output content
    if hasattr(response, "results") and response.results:
        return response.results
    elif hasattr(response, "output") and response.output:
        return response.output
    elif hasattr(response, "content"):
        return response.content
    else:
        return response  # fallback
         
def run_agent(mcp_url: str,key: str, secret: str, sessionToken: str,serviceName: str, awsRegion: str):
    mcp_client = MCPClient(lambda: create_streamable_http_transport_sigv4(mcp_url,key,secret,sessionToken,serviceName, awsRegion))

    with mcp_client:
        tools = get_full_tools_list(mcp_client)
        print(f"以下のツールが見つかりました: {[tool.tool_name for tool in tools]}")
        print(f"ツール名: {tools[0].tool_name}")
        
        
        agent = Agent(model=yourmodel,tools=tools) ## you can replace with any model you like
        print(f"エージェントに読み込まれたツール: {agent.tool_names}")
        agent("注文 ID 123 の注文状況を確認し、ツールからの正確なレスポンスを表示してください")
        #call mcp with tool
        tool=tools[0].tool_name
        tool_id="get-order-id-123-call-1"
        result = call_tool_sync(
            mcp_client,
            tool_id,
            tool_name=tool,
            parameters={"orderId": "123"}
        )

        print(f"ツール呼び出し結果: {result['content'][0]['text']}")

IAM Gateway Invoke ロールを引き受けてエージェントを実行

In [ ]:
sts_client = boto3.client("sts")
response = sts_client.assume_role(
        RoleArn=agentcore_gateway_iam_invoke_role['Role']['Arn'],
        RoleSessionName="invoke_mcp_session",
        DurationSeconds=3600  # 1 hour, can be up to 12h for some roles
)

creds = response["Credentials"]

access=creds["AccessKeyId"]
secret=creds["SecretAccessKey"]
token=creds["SessionToken"]

# Run Agent with the credentials from this new gateway-invoke-role
time.sleep(10) 
run_agent(gatewayURL,access,secret,token, SERVICE, os.environ['AWS_DEFAULT_REGION'])

**問題：以下のセルを実行中に下記のエラーが発生した場合、pydantic と pydantic-core のバージョン間の非互換性を示しています。**

```
TypeError: model_schema() got an unexpected keyword argument 'generic_origin'
```
**解決方法**

pydantic==2.7.2 と pydantic-core 2.27.2 の両方が互換性のあるバージョンであることを確認してください。完了したらカーネルを再起動してください。

# クリーンアップ

IAM ロール、IAM ポリシー、認証情報プロバイダー、AWS Lambda 関数などの追加リソースも作成されます。これらはクリーンアップの一環として手動で削除する必要がある場合があります。これは実行した例によって異なります。

## Gateway の削除（オプション）

In [ ]:
import utils
utils.delete_gateway(gateway_client,gatewayID)